TABLE OF CONTENTS:

1) Prep Data for Emrax 188 Motors

2) Prep Data for AMK Motors

3) Prep Battery Data

Plot 1: Torque vs Irms

Purpose: To determine the flux linkage of the permanent magnet in the motor

In [22]:
import numpy as np

pairs = [(50, 30), (100, 53), (150,75), (200, 90)]

TI_array = np.array(pairs)

# Now fit a slope to this line
# Matrix for linear regression
G = np.array([np.ones(len(TI_array[:,0])), TI_array[:,0]])

G = np.transpose(G)

# y-values for linear regression
y = TI_array[:,1]

# Model output
coeffs = np.linalg.lstsq(G, y, rcond=None)[0]

# Determine PM flux linkage
pm = coeffs[1] / (3/2*10)
print(pm)

0.026933333333333333


Plot 2: Motor Torque vs Speed

Version 1 - Peak Torque

Version 2 - Continuous Torque

In [23]:
pairs_peak = [(0, 90), (1000, 90), (2000, 90), (3000, 90), (4000, 87), (5000, 85), (6000, 85), (7000, 80)]
pairs_cont = [(0, 45), (1000, 47), (2000, 52), (3000, 52), (4000, 52), (5000, 52), (6000, 50), (7000, 45)]

peak_arr = np.array(pairs_peak)
cont_arr = np.array(pairs_cont)

# initialize empty arrays - Fill each space with 100 points
peak = np.empty([0, 2])
cont = np.empty([0, 2])

# number of intermediate values
intermediateVals = 100

# Then fill the spaces in between using interpolation:
for i in range(0, len(pairs_peak) - 1):
    new_peak = np.linspace(peak_arr[i],peak_arr[i+1], intermediateVals)
    new_cont = np.linspace(cont_arr[i], cont_arr[i+1], intermediateVals)
    peak = np.append(peak, new_peak, axis=0)
    cont = np.append(cont, new_cont, axis=0)

# Now print the full outputs - thinking at the moment this may be a problem because the arrays have two columns
print(peak)

# The outputs are of the following form: [rpm, torque]

[[   0.           90.        ]
 [  10.1010101    90.        ]
 [  20.2020202    90.        ]
 ...
 [6979.7979798    80.1010101 ]
 [6989.8989899    80.05050505]
 [7000.           80.        ]]


Plot 3: Efficiency Curve

Process: converted online using https://plotdigitizer.com/app

Determined efficiencies and sent to a .csv file

Now: Unpack .csv and create lookup table for efficiencies

In [24]:
import pandas as pd

infile = "emrax_efficiency_data.csv"
eff_df = pd.read_csv(infile)
print(eff_df)

# Functions as lookup table

           Speed     Torque  Efficiency
0     467.117659  19.573638        0.86
1     413.030086  32.267442        0.86
2     457.283309  44.864335        0.86
3     565.458006  55.910847        0.86
4     747.388074  67.441860        0.86
..           ...        ...         ...
102  2424.093842  47.383721        0.96
103  2699.446632  39.728679        0.96
104  2999.385747  34.302326        0.96
105  3638.598675  30.329451        0.96
106  4371.235675  32.170531        0.96

[107 rows x 3 columns]


Output data for simulation - sent to json file

In [25]:
import json

peak = peak.tolist()
cont = cont.tolist()
eff_df = eff_df.to_dict()

# Dictionary
outDict = {'lambda_pm': pm, 'PeakTorqueSpeed': peak, 'ContTorqueSpeed': cont,'Motor Efficiency': eff_df}

# Export as .json
outfile = "emrax_data.json"
with open(outfile, 'w') as export:
    export.write(json.dumps(outDict))

AMK Motor Data

Step 1: Torque-Speed Curve Data
* This is the MAXIMUM TORQUE per SPEED for both continuous and peak conditions

In [26]:
import pandas as pd

infile1 = "AMK_Peak_Torque_Speed.csv"
AMK_peak_df = pd.read_csv(infile1)

infile2 = "AMK_Cont_Torque_Speed.csv"
AMK_cont_df = pd.read_csv(infile2)

Step 2: Torque-Speed AND Current
* A more detailed plot from AMK for motor torque based on speed and current
* Conditions from AMK include: Current is varied from 0 to 105 A in steps of 5.25 A
* Motor rpm is varied from 0 to 20,000 rpm in steps of 100 rpm

In [27]:
import numpy as np

infile = "AMK_ShaftTorque.csv"
step = 5.25
col_names = np.arange(0,105+step, step)

step = 100
row_names = np.arange(0,20000+step,step)

torque_matrix = pd.read_csv(infile, names=col_names)
torque_matrix = torque_matrix.dropna(axis=0)

torque_matrix.insert(0, "Speeds", row_names)
torque_matrix.set_index('Speeds', inplace = True)

In [28]:
# Now test using this as a lookup table
# We have two options:
# 1 - Use the extra two vectors to determine the location of the information in the lookup table
# 2 - Try to search through the header and the index

# my combined solution will be transfering only the dataframe to the final code and then using these vectors to search
colNames = list(torque_matrix.columns)
rowNames = list(torque_matrix.index)

# Function for finding a close match
def findClosestMatch(vector, x):
    # vector = vector # simplifying whatever form we were given as the vector

    index = np.searchsorted(vector, x)
    # based on the return value of searchsorted. We need to check THAT and the value below (with one edge case)
    if index != 0:

        if abs(x - vector[index]) > abs(x - vector[index - 1]):
            index = index - 1
    
    return index

current = 20    # A
torque = 4     # Nm
speed = 1200    # rpm

# Practice finding the RPM based on a KNOWN CURRENT and KNOWN TORQUE
current_index = findClosestMatch(colNames, current)
t_index = findClosestMatch(torque_matrix.iloc[:,current_index].to_list(), torque)
RPM_value = rowNames[t_index]     # return RPM value

# Practice finding TORQUE based on a KNOWN RPM and KNOWN CURRENT
current_index = findClosestMatch(colNames, current)
RPM_index = findClosestMatch(rowNames, speed)
Torque_value = torque_matrix.iloc[RPM_index, current_index]

# Practice finding current based on a KNOWN RPM and KNOWN TORQUE
RPM_index = findClosestMatch(rowNames, speed)
t_index = findClosestMatch(torque_matrix.iloc[RPM_index, :].to_list(), torque)
current_value = colNames[t_index]

Step 3: Power Factor Data AND total loss Data

In [29]:
# Total loss data
infile = "AMK_TotalLosses.csv"
loss_matrix = pd.read_csv(infile, names = col_names)
loss_matrix = loss_matrix.dropna(axis=0)

loss_matrix.insert(0, "Speeds", row_names)
loss_matrix.set_index('Speeds', inplace = True)

# Power Factor Data
infile = "AMK_PowerFactor.csv"
pf_matrix = pd.read_csv(infile, names = col_names)
pf_matrix = pf_matrix.dropna(axis=0)

pf_matrix.insert(0, "Speeds", row_names)
pf_matrix.set_index('Speeds', inplace = True)

# This could also be expanded to Temperature, etc.

Export Data as JSON file

In [30]:
import json

torque_matrix = torque_matrix.to_dict()
pf_matrix = pf_matrix.to_dict()
loss_matrix = loss_matrix.to_dict()
AMK_peak_df = AMK_peak_df.to_dict()
AMK_cont_df = AMK_cont_df.to_dict()

# Combine data into a dictionary
outDict = {'ShaftTorque': torque_matrix, 'PowerFactor': pf_matrix, 'TotalLosses': loss_matrix, 'PeakTorqueSpeed': AMK_peak_df, "ContTorqueSpeed": AMK_cont_df}

# Export as .json
outfile = "AMK_data.json"
with open(outfile, 'w') as export:
    export.write(json.dumps(outDict))

In [5]:
import numpy as np

like = np.array([1, 2, 3])

a1 = np.array([1, 2, 3, 4, 5])
a2 = np.array([1, 2])
a3 = np.array([1, 2, 3, 4])

# atotal = np.empty_like(like)

# atotal[0] = a1
# atotal[1] = a2
# atotal[2] = a3

# ARRAY UNSUCCESSFUL

# Check dict
dtotal = dict({'a1': a1, 'a2': a2, 'a3': a3})
keylist = list(dtotal.keys())

# This will work and I will use the key list to iterate through the dict


# Can we put a dataframe into a dict?
# Import pandas library
import pandas as pd

# initialize list of lists
data = [['tom', 10], ['nick', 15], ['juli', 14]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

dtotal['a1'] = df # answer is yes!

: 